In [ ]:
import pandas as pd
import numpy as py

# 1. Aquisição de dados

In [ ]:
df2 = pd.read_csv("/content/drive/MyDrive/Pós-Graduação/Mineração de Dados/titanic_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Pós-Graduação/Mineração de Dados/titanic_test.csv")

# 2. Pré-processamento

In [ ]:
df2

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
# Função p/ transformar dados string para integer

def transformar(x):
  if x == 'male':
    return 0
  else:
    return 1

In [ ]:
def transformar2(x):
  if x == 'S':
    return 0
  elif x == 'C':
    return 1
  else:
    return 2

In [ ]:
df2["Embarked_2"] = df2["Embarked"].map(transformar2)

In [ ]:
df2["Category"] = df2["Sex"].map(transformar)

In [ ]:
df2

In [ ]:
df3 = df2.drop(columns=['Embarked','Name','Ticket','Cabin'])

In [ ]:
df3

In [ ]:
# Visualizar os tipos de categorias

df3.dtypes

In [ ]:
# Separar Label e Features

label_df = df3["Survived"]

feat_cols = df3.columns.tolist()
feat_cols.remove("Survived")

features_df = df3[feat_cols]
features_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_2
0,1,3,male,22.0,1,0,7.2500,0
1,2,1,female,38.0,1,0,71.2833,1
2,3,3,female,26.0,0,0,7.9250,0
3,4,1,female,35.0,1,0,53.1000,0
4,5,3,male,35.0,0,0,8.0500,0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_df, label_df, test_size=0.33, random_state = 42)

In [ ]:
from sklearn.preprocessing import LabelEncoder

category_cols = ["Sex"]


# Label encode categorical features
le = {}
for col in category_cols:
    le[col] = LabelEncoder()
    X_train[col] = le[col].fit_transform(X_train[col])

In [ ]:
X_train.head()

In [ ]:
X_train.fillna(0.0, inplace = True)

In [ ]:
# Padronização dos dados

from sklearn.preprocessing import StandardScaler
se = StandardScaler()
X_train = se.fit_transform(X_train)

In [ ]:
# Verificar quantidade de dados positivos (dados estão desbalanceados)

y_train.sum() / y_train.shape[0]

In [ ]:
# Balanceamento dos dados

from imblearn.over_sampling import SMOTE

oversample = SMOTE()
X_, y_ = oversample.fit_resample(X_train, y_train)

In [ ]:
import numpy as np

print("Dados desbalanceados:",np.bincount(y_train))
print("Dados balanceados:",np.bincount(y_))

# 3. Criação do Modelo (XGBoost)

In [ ]:
import xgboost as xgb

clf = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=15
)
clf.fit(X_, y_)

# 4. Validação do Modelo

In [ ]:
X_test

In [ ]:

for col in category_cols:
    X_test[col] = le[col].transform(X_test[col])
X_test.fillna(0.0, inplace=True)
X_test = se.transform(X_test)

In [ ]:
from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(clf, X_test, y_test, normalize="true")

# 5. Salvar para implantar

In [ ]:
import pickle

with open("model.pkl", "wb") as handle:
    pickle.dump({"model": clf, "se": se, "le":le}, handle)

In [ ]:
with open("model.pkl", "rb") as handle:
    data = pickle.load(handle)